In [3]:
#Polynomial matrix:

import numpy as np
import mpmath as mp

# Pauli matrices
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)
I = np.eye(2, dtype=complex)

# Coefficients from Bessel functions
def C0(t):
    return mp.besselj(0, t) + 2*mp.besselj(2, t) + 2*mp.besselj(4, t)

def C2(t):
    return -4*mp.besselj(2, t) - 16*mp.besselj(4, t)

def C4(t):
    return 16*mp.besselj(4, t)

# Build H
def build_H(J1, J2):
    return J1 * X + J2 * Z

# Polynomial in H
def polynomial_H(t, J1, J2):
    H = build_H(J1, J2)
    H2 = np.matmul(H, H)
    H4 = np.matmul(H2, H2)
    P = C0(t)*I + C2(t)*H2 + C4(t)*H4
    return np.array(P, dtype=complex)

# Example usage
t = 1.6
J1, J2 = 0.25, 0.4
P = polynomial_H(t, J1, J2)
print("Polynomial P(t, J1, J2) =\n", P)

Polynomial P(t, J1, J2) =
 [[0.66914094+0.j 0.        +0.j]
 [0.        +0.j 0.66914094+0.j]]


In [3]:
#Polynomial matrix:

import numpy as np
import mpmath as mp

# Pauli matrices
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)
I = np.eye(2, dtype=complex)

# Coefficients from Bessel functions
def C0(t):
    return mp.besselj(0, t) + 2*mp.besselj(2, t) + 2*mp.besselj(4, t)

def C2(t):
    return -4*mp.besselj(2, t) - 16*mp.besselj(4, t)

def C4(t):
    return 16*mp.besselj(4, t)

# Build H
def build_H(J1, J2):
    return J1 * X + J2 * Z

# Polynomial in H
def polynomial_H(t, J1, J2):
    H = build_H(J1, J2)
    H2 = np.matmul(H, H)
    H4 = np.matmul(H2, H2)
    P = C0(t)*I + C2(t)*H2 + C4(t)*H4
    return np.array(P, dtype=complex)

# Example usage
t = 1.6
J1, J2 = 0.25, 0.4
P = polynomial_H(t, J1, J2)
print("Polynomial P(t, J1, J2) =\n", P)

Polynomial P(t, J1, J2) =
 [[0.66914094+0.j 0.        +0.j]
 [0.        +0.j 0.66914094+0.j]]


In [2]:
#Apply on [1,0]:


import numpy as np
import mpmath as mp
from scipy.special import jv

# Pauli matrices
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)
I = np.eye(2, dtype=complex)

# Coefficients from Bessel functions
def C0(t):
    return mp.besselj(0, t) + 2*mp.besselj(2, t) + 2*mp.besselj(4, t)

def C2(t):
    return -4*mp.besselj(2, t) - 16*mp.besselj(4, t)

def C4(t):
    return 16*mp.besselj(4, t)

# Build H
def build_H(J1, J2):
    total = 2 * J1 + J2
    J1 = J1/total
    J2 = J2/total
    return J1 * np.kron(X,I) + J1 * np.kron(I,X) + J2 * np.kron(Z,Z)

# Polynomial in H
def polynomial_H(t, J1, J2, max_degree):
    H = build_H(J1, J2)
    # H2 = H @ H
    # H4 = H2 @ H2
    # P = C0(t)*np.kron(I,I) + C2(t)*H2 + C4(t)*H4
    coeff_total = np.zeros(2 * max_degree)
    T = np.zeros((2 * max_degree, 2 * max_degree))
    T[0][0] = 1
    T[1][1] = 1
    for i in range(2, len(T)):
        T[i,:] = 2 * np.concatenate((np.zeros(1),T[i-1,0:len(T[0])-1]))
        T[i,:] -= T[i-2,:]
    coeff_total = jv(0, t) * T[0, :]
    for k in range(2, len(T), 2):
        coeff_total += 2 * (-1)**(int(k/2)) * jv(k, t) * T[k, :] 
    temp = np.eye(len(H))
    polynomial = np.zeros((len(H),len(H)), dtype = complex)
    for i in range(len(coeff_total)):
        polynomial += temp * coeff_total[i]
        temp = H @ temp
    return polynomial
   

# Apply polynomial on vector (1,0)
def apply_polynomial(t, J1, J2, max_degree):
    # P = build_H(J1, J2)
    P = polynomial_H(t,J1,J2, max_degree)
    print("resultant matrix =\n", P)

    

    v = np.array([[1],[0],[0],[0]], dtype=complex)
    result = P @ v
    return result

# Example usage
t = 0.8
J1, J2 = 0.25, 0.4
max_degree = 3
result = apply_polynomial(t, J1, J2, max_degree)
print("Resulting vector =\n", result)



resultant matrix =
 [[ 8.89906784e-01+0.j  5.73421862e-20+0.j  1.14684372e-19+0.j
  -4.75568154e-02+0.j]
 [-5.73421862e-20+0.j  8.89906784e-01+0.j -4.75568154e-02+0.j
  -1.14684372e-19+0.j]
 [-5.73421862e-20+0.j -4.75568154e-02+0.j  8.89906784e-01+0.j
  -1.14684372e-19+0.j]
 [-4.75568154e-02+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   8.89906784e-01+0.j]]
Resulting vector =
 [[ 8.89906784e-01+0.j]
 [-5.73421862e-20+0.j]
 [-5.73421862e-20+0.j]
 [-4.75568154e-02+0.j]]


In [1]:
import numpy as np
import mpmath as mp

# Pauli matrices
X = np.array([[0,1],[1,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
I = np.eye(2, dtype=complex)

# Bessel-based coefficients
def C0(t):
    return float(mp.besselj(0, t) + 2*mp.besselj(2, t) - 2*mp.besselj(4, t))
def C2(t):
    return float(-4*mp.besselj(2, t) - 16*mp.besselj(4, t))
def C4(t):
    return float(16*mp.besselj(4, t))

# Build H and polynomial P
def build_H(J1, J2):
    return J1*X + J2*Z

def polynomial_P(t, J1, J2):
    H = build_H(J1, J2)
    H2 = H @ H
    H4 = H2 @ H2
    P = C0(t)*I + C2(t)*H2 + C4(t)*H4
    return np.array(P, dtype=complex)

# basis states
zero_state = np.array([1,0], dtype=complex)
plus_state = np.array([1,1], dtype=complex)/np.sqrt(2)

# Apply to first qubit and build n-qubit state
def apply_to_first_qubit_and_build_full(t, J1, J2, n_qubits):
    P = polynomial_P(t, J1, J2)
    single_out = P @ zero_state
    single_out = single_out / np.linalg.norm(single_out)
    full_state = single_out.copy()
    for _ in range(n_qubits-1):
        full_state = np.kron(full_state, zero_state)
    return full_state

# Probability of measuring |+ 0 0 ... 0>
def probability_first_plus_rest_zero(t, J1, J2, n_qubits):
    full = apply_to_first_qubit_and_build_full(t, J1, J2, n_qubits)
    target = plus_state.copy()
    for _ in range(n_qubits-1):
        target = np.kron(target, zero_state)
    full = full / np.linalg.norm(full)
    target = target / np.linalg.norm(target)
    amplitude = np.vdot(target, full)
    prob = abs(amplitude)**2
    return prob, amplitude, full, target

# Example
t = 2.8
J1 = 0.25
J2 = 0.4
n_qubits = 3

prob, amp, full_state, target_state = probability_first_plus_rest_zero(t, J1, J2, n_qubits)
print("Probability =", prob)
print("Amplitude =", amp)

Probability = 0.5000000000000001
Amplitude = (-0.7071067811865476+0j)


In [ ]:
# cosine part

In [4]:
import numpy as np
from scipy.special import jv
from numpy.polynomial.chebyshev import Chebyshev
from scipy.optimize import minimize
from functools import reduce

# ----- Symbolic Polynomial Algebra -----

def symbolic_poly_multiply(p1, p2):
    result_len = len(p1) + len(p2) - 1
    result = [0j] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            result[i + j] += p1[i] * p2[j]
    return result

def symbolic_poly_multiply_offdiagonal(p1, p2):
    result_len = len(p1) + len(p2) + 1
    result = [0j] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            term = p1[i] * p2[j]
            result[i + j + 1] += term
            result[i + j + 2] -= term
    return result

def add_symbolic_polynomials(p1, p2):
    max_len = max(len(p1), len(p2))
    p1 += [0j] * (max_len - len(p1))
    p2 += [0j] * (max_len - len(p2))
    return [a + b for a, b in zip(p1, p2)]

# ----- Symbolic Matrix Operations -----

def multiply_symbolic_matrices(A, B):
    R = np.empty((2, 2), dtype=object)
    for i in range(2):
        for j in range(2):
            result = [0j]
            for k in range(2):
                if i != k and j != k:
                    prod = symbolic_poly_multiply_offdiagonal(B[i, k], A[k, j])
                else:
                    prod = symbolic_poly_multiply(B[i, k], A[k, j])
                result = add_symbolic_polynomials(result, prod)
            R[i, j] = result
    return R

# ----- Generate 2x2 Symbolic Matrix from f -----

def create_numeric_matrix(f, idx):
    A = np.empty((2, 2), dtype=object)

    f1, f2 = f[2 * idx], f[2 * idx + 1]
    barf1 = np.conj(f1)

    a00 = barf1 * f2
    a01 = f1 * f2 - barf1 * f2
    a10 = a01
    a20 = -np.conj(a01)
    a30 = np.conj(a00)
    a31 = np.conj(a01)

    A[0, 0] = [a00, a01]
    A[0, 1] = [a10]
    A[1, 0] = [a20]
    A[1, 1] = [a30, a31]
    return A

# ----- Target Coefficients from Chebyshev Series -----

def target_coeffs(t, max_k, max_degree):
    coeff_total = np.zeros(2 * max_degree)
    T = np.zeros((2 * max_degree, 2 * max_degree))
    T[0][0] = 1
    T[1][1] = 1
    for i in range(2, len(T)):
        T[i,:] = 2 * np.concatenate((np.zeros(1),T[i-1,0:len(T[0])-1]))
        T[i,:] -= T[i-2,:]
    coeff_total = jv(0, t) * T[0, :]
    for k in range(2, len(T), 2):
    #     n = 2 * k
    #     J_val = jv(n, t)
    #     factor = (-1) ** k * 2 * J_val

        # # Get Chebyshev T_{2k}(x), convert to power basis
        # Tn = Chebyshev.basis(n)
        # coeffs = Tn.convert(kind=np.polynomial.Polynomial).coef
        # padded = np.pad(coeffs, (0, max_degree + 1 - len(coeffs)))

        # coeff_total += factor * padded
        coeff_total += 2 * (-1)**(int(k/2)) * jv(k, t) * T[k, :] 
    return coeff_total

# ----- Cost Function -----

def compute_cost(phases, t):
    # Create f_i = exp(2j * phi_i)
    f = [np.exp(2j * phi) for phi in phases]

    # Create symbolic matrices
    matrices = [create_numeric_matrix(f, i) for i in range(len(f) // 2)]

    # Multiply all matrices: R = A0 * A1 * A2 * ...
    R = reduce(multiply_symbolic_matrices, matrices)

    # Extract polynomial from R[0,0]
    R00 = R[0, 0]

    # Get target coefficients
    target = target_coeffs(t, len(R00)-1, (len(R00) - 1) * 2)
    target = target[0::2]
    # print(R00)
    # print(target)

    # Compute cost: sum of squared real-part differences for even powers only
    cost = 0.0
    for i in range(0, min(len(R00), len(target))):
        cost += (R00[i].real - target[i]) ** 2
    return cost

def compute_cost_Test(phases, t):
    # Create f_i = exp(2j * phi_i)
    f = [np.exp(1j * phi) for phi in phases]

    # Create symbolic matrices
    matrices = [create_numeric_matrix(f, i) for i in range(len(f) // 2)]

    # Multiply all matrices: R = A0 * A1 * A2 * ...
    R = reduce(multiply_symbolic_matrices, matrices)

    # Extract polynomial from R[0,0]
    R00 = R[0, 0]
    
    # Get target coefficients
    target = target_coeffs(t, len(R00)-1, (len(R00) - 1) * 2)
    target = target[0::2]
    print(R00)
    print(target)

    # Compute cost: sum of squared real-part differences for even powers only
    cost = 0.0
    for i in range(0, min(len(R00), len(target))):
        cost += (R00[i].real - target[i]) ** 2
    return cost


# ----- Run Optimization -----

if __name__ == "__main__":
    # Parameters
    t = 0.8
    num_phases = 4 # 3 matrices => 6 phases

    # Initial guess for phases
    initial_phases = np.random.rand(num_phases)
    # Minimize cost
    result = minimize(compute_cost, initial_phases, args=(t), method='BFGS')

    # Output results
    print("Optimized phases:")
    print(result.x)
    print("\nMinimum cost:")
    print(result.fun)

    compute_cost_Test(result.x, t)

Optimized phases:
[-1.56797236  2.21882546  1.36757082  7.00660066]

Minimum cost:
4.778450391352084e-12
[(-0.9999994490667361-0.0010496981586599063j), (4.213979102031927-4.292130719353523j), (-4.135136658746877+4.68237159251403j)]
[ 9.99999968e-01 -3.19998975e-01  1.70615343e-02 -3.55849586e-04]
